### LSE Data Analytics Online Career Accelerator

# DA201: Data Analytics using Python

## Practical activity: Search the OpenWeather API

**Important**

Please take note that you will work with the OpenWeather API. Keep in mind that the API is based on live and current weather data. Therefore, your output will differ from the outputs provided. 

**This is the solution to the activity.**

You have recently had a discussion with a group of people about the weather. Many of you recalled childhood summers and winters that seemed less extreme than they are now. As you now know how to retrieve and compare current and historical data using the OpenWeather API, you have decided to explore and compare weather data for multiple locations across a range of years.

__Prepare__: 
-  Make a list of the locations you would like to obtain weather data for.
-  Decide on the years you would like to explore for comparison, and convert them to Unix timestamps for use in your OpenWeather API call. 

__Start__
1. Import the libraries and access your key.
4. Check your connection to the API. Confirm the connection by making a basic API call.
5. Obtain the coordinates for your selected locations. 
6. Design an API call with your chosen parameters.
7. Print the output with json (and json.dumps() if required).
8. Create formatted DataFrame/s that enable ease of comparison across locations and years.
1. Draw a conclusion
9. Export your combined DataFrame/s to CSV.

__Finish__:

Save the Notebook to your GitHub repository by uploading and committing any changes.

## 1. Prepare your workstation

In [1]:
# Import the necessary libraries before you start to work.
import os
import requests
import pandas as pd
import json
import pytz
from datetime import datetime
from dotenv import load_dotenv

load_dotenv()

# Access the API key.
API_key = os.getenv('OpenWeather_API_key')

In [2]:
# See if you are connected.
# Create an API call.
weather_url = "https://api.openweathermap.org/data/3.0/onecall?lat=51.509865&lon=-0.118092&exclude=minutely,hourly,daily" "&appid=" + API_key

# Define a response variable.
resp = requests.get(weather_url)

# Print the response.
print("Today's weather in London:" , resp.json())

Today's weather in London: {'lat': 51.5099, 'lon': -0.1181, 'timezone': 'Europe/London', 'timezone_offset': 3600, 'current': {'dt': 1760915791, 'sunrise': 1760941974, 'sunset': 1760979436, 'temp': 287.42, 'feels_like': 287.23, 'pressure': 995, 'humidity': 89, 'dew_point': 285.63, 'uvi': 0, 'clouds': 75, 'visibility': 10000, 'wind_speed': 5.66, 'wind_deg': 220, 'weather': [{'id': 803, 'main': 'Clouds', 'description': 'broken clouds', 'icon': '04n'}]}}


## 2. Obtain coordinates for selected locations and create a list

In [3]:
# Set the API endpoint URL and API key.
endpoint_url = "http://api.openweathermap.org/geo/1.0/direct"
API_key = os.environ.get('OpenWeather_API_key')

# Define a list of different locations
locations = [
    {'name': 'London'},
    {'name': 'Paris'},
    {'name': 'New York'},
    {'name': 'Tokyo'},
    {'name': 'Sydney'},
    {'name': 'Shanghai'}, 
]

# Create an empty list to store the coordinates for each location.
coordinates = []

# Loop through each location and get its coordinate data.
for loc in locations:
    # Set the API parameters to search for the location by name.
    params = {
        "q": loc['name'],
        "appid": API_key
    }
     
    # Send a GET request to the API endpoint and store the response.
    response = requests.get(endpoint_url, params=params)
    
    # Extract the JSON data from the response.
    data = response.json()
    
    # Extract latitude and longitude coordinates and append to the list.
    if data:
        latitude = data[0]["lat"]
        longitude = data[0]["lon"]
        coordinates.append({'name': loc['name'], 'lat': latitude, 'lon': longitude})

# Print the list of coordinates for each location.
for coord in coordinates:
    print(f"{coord['name']}: Latitude = {coord['lat']}, Longitude = {coord['lon']}")

# Use the coordinates list to create the locations list.
locations = [{'name': coord['name'], 'lat': coord['lat'], 'lon': coord['lon']} for coord in coordinates]

# Print the locations list.
print(locations)

London: Latitude = 51.5073219, Longitude = -0.1276474
Paris: Latitude = 48.8588897, Longitude = 2.3200410217200766
New York: Latitude = 40.7127281, Longitude = -74.0060152
Tokyo: Latitude = 35.6828387, Longitude = 139.7594549
Sydney: Latitude = -33.8698439, Longitude = 151.2082848
Shanghai: Latitude = 31.2322758, Longitude = 121.4692071
[{'name': 'London', 'lat': 51.5073219, 'lon': -0.1276474}, {'name': 'Paris', 'lat': 48.8588897, 'lon': 2.3200410217200766}, {'name': 'New York', 'lat': 40.7127281, 'lon': -74.0060152}, {'name': 'Tokyo', 'lat': 35.6828387, 'lon': 139.7594549}, {'name': 'Sydney', 'lat': -33.8698439, 'lon': 151.2082848}, {'name': 'Shanghai', 'lat': 31.2322758, 'lon': 121.4692071}]


### Explanation: 

-The API parameters search for the location by name, using the q parameter.

-Inside the loop, the latitude and longitude coordinates are extracted from the JSON response using the data variable, and appending a dictionary containing the location name, latitude, and longitude to the coordinates list.

-The list of coordinates for each location is then printed using a for loop.

-The additional loop after the previous loop creates the locations list based on the coordinates list that was created. This loop uses a list comprehension to create the locations list in the correct format for making multiple API calls to obtain weather data for each location.

-The locations list is printed, ready for use.

## 3. Obtain the current weather data for multiple locations

In [4]:
# Set the API endpoint URL and API key.
endpoint_url = "https://api.openweathermap.org/data/3.0/onecall"

#Access your API key.
API_key = os.environ.get('OpenWeather_API_key')

# Define a list of coordinates for different locations
locations = [
    {'name': 'London', 'lat': 51.5073219, 'lon': -0.1276474},
    {'name': 'Paris', 'lat': 48.8588897, 'lon': 2.3200410217200766}, 
    {'name': 'New York', 'lat': 40.7127281, 'lon': -74.0060152}, 
    {'name': 'Tokyo', 'lat': 35.6828387, 'lon': 139.7594549}, 
    {'name': 'Sydney', 'lat': -33.8698439, 'lon': 151.2082848}, 
    {'name': 'Shanghai', 'lat': 31.2322758, 'lon': 121.4692071}]

# Create an empty list to store the weather data for each location.
weather_data = []

# Loop through each location and get its weather data.
for loc in locations:
    # Set the API parameters.
    params = {
        "lat": loc['lat'],
        "lon": loc['lon'],
        "units": "metric",
        "exclude": "minutely,hourly,daily",
        "appid": API_key
    }

    # Send a GET request to the API endpoint and store the response.
    response = requests.get(endpoint_url, params=params)

    # Access the current weather data.
    current = response.json()['current']

    # Extract the relevant weather information.
    humidity = current['humidity']
    pressure = current['pressure']
    wind = current['wind_speed']
    description = current['weather'][0]['description']
    temp = int(round(current['temp']))
    sunrise = datetime.fromtimestamp(current['sunrise'], tz=pytz.utc).astimezone(pytz.timezone('Europe/London'))
    sunset = datetime.fromtimestamp(current['sunset'], tz=pytz.utc).astimezone(pytz.timezone('Europe/London'))

    # Add the weather data to the list.
    weather_data.append({
        'Location': loc['name'],
        'Temperature (°C)': temp,
        'Wind Speed (m/s)': wind,
        'Pressure (hPa)': pressure,
        'Humidity (%)': humidity,
        'Description': description,
        'Sunrise': sunrise,
        'Sunset': sunset
    })

# Print the weather data in a tabular format.
print("Location\tTemperature (°C)\tWind Speed (m/s)\tPressure (hPa)\tHumidity (%)\tDescription\tSunrise\t\t\t\tSunset")
for data in weather_data:
    print(f"{data['Location']}\t\t{data['Temperature (°C)']}\t\t{data['Wind Speed (m/s)']}\t\t{data['Pressure (hPa)']}\t\t{data['Humidity (%)']}\t\t{data['Description']}\t{data['Sunrise'].strftime('%Y-%m-%d %H:%M:%S')}\t{data['Sunset'].strftime('%Y-%m-%d %H:%M:%S')}")
    

Location	Temperature (°C)	Wind Speed (m/s)	Pressure (hPa)	Humidity (%)	Description	Sunrise				Sunset
London		14		5.66		995		88		broken clouds	2025-10-20 07:32:56	2025-10-20 17:57:19
Paris		15		7.2		1002		93		mist	2025-10-20 07:18:36	2025-10-20 17:52:05
New York		18		9.83		1010		84		overcast clouds	2025-10-19 12:11:36	2025-10-19 23:09:57
Tokyo		16		3.6		1018		89		heavy intensity rain	2025-10-19 21:51:20	2025-10-20 08:59:57
Sydney		27		1.34		1011		52		clear sky	2025-10-19 20:07:52	2025-10-20 09:11:50
Shanghai		18		8		1026		63		broken clouds	2025-10-19 23:00:05	2025-10-20 10:17:31


## 4. Create a DataFrame for current weather

In [5]:
# Create a data frame from the list of dictionaries.
df1 = pd.DataFrame(weather_data)

styled_df = df1.style \
    .set_caption("Example DataFrame") \
    .set_properties(**{'text-align': 'center'}) \
    .set_table_styles([{'selector': 'th', 'props': [('text-align', 'center')]}]) \
    .background_gradient(cmap='viridis', low=0, high=1)

# display the styled DataFrame
display(styled_df)

# export dataframe to csv
df1.to_csv('my_current_data.csv', index=False)

,Location,Temperature (°C),Wind Speed (m/s),Pressure (hPa),Humidity (%),Description,Sunrise,Sunset
0,London,14,5.660000,995,88,broken clouds,2025-10-20 07:32:56+01:00,2025-10-20 17:57:19+01:00
1,Paris,15,7.200000,1002,93,mist,2025-10-20 07:18:36+01:00,2025-10-20 17:52:05+01:00
2,New York,18,9.830000,1010,84,overcast clouds,2025-10-19 12:11:36+01:00,2025-10-19 23:09:57+01:00
3,Tokyo,16,3.600000,1018,89,heavy intensity rain,2025-10-19 21:51:20+01:00,2025-10-20 08:59:57+01:00
4,Sydney,27,1.340000,1011,52,clear sky,2025-10-19 20:07:52+01:00,2025-10-20 09:11:50+01:00
5,Shanghai,18,8.000000,1026,63,broken clouds,2025-10-19 23:00:05+01:00,2025-10-20 10:17:31+01:00


## 5. Obtain historical data for the selected locations for 1983 - timestamp 420994240

In [6]:
import requests
import os
from datetime import datetime
import pytz
import pandas as pd

# Set the API endpoint URL and API key.
# Create an API call for the historical weather in your locations.
endpoint_url = "https://api.openweathermap.org/data/3.0/onecall/timemachine?dt=420994240"
#Access your API key.
API_key = os.environ.get('OpenWeather_API_key')


# Define a list of coordinates for different locations
locations = [
    {'name': 'London', 'lat': 51.5073219, 'lon': -0.1276474},
    {'name': 'Paris', 'lat': 48.8588897, 'lon': 2.3200410217200766}, 
    {'name': 'New York', 'lat': 40.7127281, 'lon': -74.0060152}, 
    {'name': 'Tokyo', 'lat': 35.6828387, 'lon': 139.7594549}, 
    {'name': 'Sydney', 'lat': -33.8698439, 'lon': 151.2082848}, 
    {'name': 'Shanghai', 'lat': 31.2322758, 'lon': 121.4692071}
]

# Create an empty list to store the weather data for each location.
hist_weather_data = []

# Loop through each location and get its weather data.
for loc in locations:
    # Set the API parameters.
    params = {
        "lat": loc['lat'],
        "lon": loc['lon'],
        "units": "metric",
        "exclude": "minutely,hourly,daily",
        "appid": API_key
    }

    # Send a GET request to the API endpoint and store the response.
    response = requests.get(endpoint_url, params=params)


    # Access the historical weather data.
    data = response.json()['data'][0]

    # Extract the relevant weather information.
    humidity =  data['humidity']
    pressure =  data['pressure']
    wind =  data['wind_speed']
    description =  data['weather'][0]['description']
    temp = int(round( data['temp']))
    sunrise = datetime.fromtimestamp(data['sunrise'], tz=pytz.utc).astimezone(pytz.timezone('Europe/London'))
    sunset = datetime.fromtimestamp(data['sunset'], tz=pytz.utc).astimezone(pytz.timezone('Europe/London'))

    # Add the weather data to the list.
    hist_weather_data.append({
        'Location': loc['name'],
        'Temperature (°C)': temp,
        'Wind Speed (m/s)': wind,
        'Pressure (hPa)': pressure,
        'Humidity (%)': humidity,
        'Description': description,
        'Sunrise': sunrise,
        'Sunset': sunset
    })

# Print the weather data in a tabular format.
print("Location\tTemperature (°C)\tWind Speed (m/s)\tPressure (hPa)\tHumidity (%)\tDescription\tSunrise\t\t\t\tSunset")
for data in hist_weather_data:
    print (f"{data['Location']}\t\t{data['Temperature (°C)']}\t\t{data['Wind Speed (m/s)']}\t\t{data['Pressure (hPa)']}\t\t{data['Humidity (%)']}\t\t{data['Description']}\t\t\{data['Sunrise'].strftime('%Y-%m-%d %H:%M:%S')}\t{data['Sunset'].strftime('%Y-%m-%d %H:%M:%S')}")
    

<string>:69: SyntaxWarning: invalid escape sequence '\{'
<>:69: SyntaxWarning: invalid escape sequence '\{'
<string>:69: SyntaxWarning: invalid escape sequence '\{'
<>:69: SyntaxWarning: invalid escape sequence '\{'
/var/folders/fg/20dz6rq139zbhlk5rp3m_rfc0000gn/T/ipykernel_60638/1521524024.py:69: SyntaxWarning: invalid escape sequence '\{'
  print (f"{data['Location']}\t\t{data['Temperature (°C)']}\t\t{data['Wind Speed (m/s)']}\t\t{data['Pressure (hPa)']}\t\t{data['Humidity (%)']}\t\t{data['Description']}\t\t\{data['Sunrise'].strftime('%Y-%m-%d %H:%M:%S')}\t{data['Sunset'].strftime('%Y-%m-%d %H:%M:%S')}")


Location	Temperature (°C)	Wind Speed (m/s)	Pressure (hPa)	Humidity (%)	Description	Sunrise				Sunset
London		16		5.18		1015		69		overcast clouds		\1983-05-05 05:25:13	1983-05-05 20:29:07
Paris		20		5.1		1014		48		scattered clouds		\1983-05-05 05:23:53	1983-05-05 20:10:52
New York		18		3.41		1009		54		scattered clouds		\1983-05-05 10:49:39	1983-05-06 00:55:41
Tokyo		16		9.7		1019		87		clear sky		\1983-05-04 20:45:21	1983-05-05 10:29:57
Sydney		14		4.78		1019		71		clear sky		\1983-05-05 21:33:07	1983-05-06 08:10:27
Shanghai		20		1.44		1009		98		moderate rain		\1983-05-04 22:06:15	1983-05-05 11:35:21


## 6. Create a DataFrame for historical weather

In [7]:
# Create a data frame from the list of dictionaries.
df2 = pd.DataFrame(hist_weather_data)

styled_df = df2.style \
    .set_caption("Example DataFrame") \
    .set_properties(**{'text-align': 'center'}) \
    .set_table_styles([{'selector': 'th', 'props': [('text-align', 'center')]}]) \
    .background_gradient(cmap='viridis', low=0, high=1)

# display the styled DataFrame
display(styled_df)

# export dataframe to csv
df2.to_csv('my_hist_data.csv', index=False)

,Location,Temperature (°C),Wind Speed (m/s),Pressure (hPa),Humidity (%),Description,Sunrise,Sunset
0,London,16,5.180000,1015,69,overcast clouds,1983-05-05 05:25:13+01:00,1983-05-05 20:29:07+01:00
1,Paris,20,5.100000,1014,48,scattered clouds,1983-05-05 05:23:53+01:00,1983-05-05 20:10:52+01:00
2,New York,18,3.410000,1009,54,scattered clouds,1983-05-05 10:49:39+01:00,1983-05-06 00:55:41+01:00
3,Tokyo,16,9.700000,1019,87,clear sky,1983-05-04 20:45:21+01:00,1983-05-05 10:29:57+01:00
4,Sydney,14,4.780000,1019,71,clear sky,1983-05-05 21:33:07+01:00,1983-05-06 08:10:27+01:00
5,Shanghai,20,1.440000,1009,98,moderate rain,1983-05-04 22:06:15+01:00,1983-05-05 11:35:21+01:00


## 7. Combine the DataFrames for comparison

In [8]:
weather_comparison = pd.merge(df1[['Location','Temperature (°C)']], df2[['Location','Temperature (°C)']],
                           how='outer', on='Location', suffixes=('_Current', '_Past'))

styled_df = weather_comparison.style \
    .set_caption("Comparison") \
    .set_properties(**{'text-align': 'center'}) \
    .set_table_styles([{'selector': 'th', 'props': [('text-align', 'center')]}]) \
    .background_gradient(cmap='viridis', low=0, high=1)

# display the styled DataFrame
display(styled_df)
# export dataframe to csv
weather_comparison.to_csv('combined_weather_data.csv', index=False)

,Location,Temperature (°C)_Current,Temperature (°C)_Past
0,London,14,16
1,New York,18,18
2,Paris,15,20
3,Shanghai,18,20
4,Sydney,27,14
5,Tokyo,16,16


## 9. Draw a conclusion- are there any significant differences in weather conditions now and in 1983?

Paris and New York were warmer in 1983, London was a degree cooler and Sydney a degree warmer, and Tokyo and Shanghai were the same in the past as now. A possible conclusion could be that there is not enough data to establish whether the temperatures today are any different from 1983. Perhaps adding more years and more cities would allow for a more meaningful comparison.